In [210]:
import fitz
import pandas as pd
from pathlib import Path
import csv

## Files management

In [166]:
def find_files(directory, pattern):
    path = Path(directory)
    return path.rglob(pattern)

# Example usage
directory = "./pdf_files"
pattern = "*.pdf"

file_paths = [i for i in find_files(directory, pattern) 
              if i.name.startswith('2')
              ]
print(*file_paths, sep='\n')

pdf_files/2005 - Reacciones Redox.pdf
pdf_files/2008 - Reacciones Redox.pdf
pdf_files/2022 - Reacciones Redox.pdf
pdf_files/2000 - Reacciones Redox.pdf
pdf_files/2003 - Reacciones Redox.pdf
pdf_files/2004 - Reacciones Redox.pdf
pdf_files/2002 - Reacciones Redox.pdf
pdf_files/2016 - Reacciones Redox.pdf
pdf_files/2021 - Reacciones Redox.pdf
pdf_files/2024 - Reacciones Redox.pdf
pdf_files/2007 - Reacciones Redox.pdf
pdf_files/2023 - Reacciones Redox.pdf
pdf_files/2015 - Reacciones Redox.pdf
pdf_files/2013 - Reacciones Redox.pdf
pdf_files/2011 - Reacciones Redox.pdf
pdf_files/2001 - Reacciones Redox.pdf
pdf_files/2014 - Reacciones Redox.pdf
pdf_files/2019 - Reacciones Redox.pdf
pdf_files/2017 - Reacciones Redox.pdf
pdf_files/2018 - Reacciones Redox.pdf
pdf_files/2020 - Reacciones Redox.pdf
pdf_files/2009 - Reacciones Redox.pdf
pdf_files/2012 - Reacciones Redox.pdf
pdf_files/2010 - Reacciones Redox.pdf
pdf_files/2006 - Reacciones Redox.pdf


In [ ]:
def extract_text_from_pdf(pdf_path, output_path=None):
    """Extracts text from a PDF and optionally saves it to a text file."""
    doc = fitz.open(pdf_path)
    

    if output_txt_path:
        with open(output_txt_path, "w", encoding="utf-8") as f:
            f.write(text)
    
    return text

In [211]:
def save_dict_to_csv(data, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        # Write the header
        writer.writerow(data.keys())
        
        # Write the values
        writer.writerow(data.values())

In [225]:
def extract_info(file_path):
    
    subjects = []
    years = []
    exams = []
    exercises = []
    statements = []
    pages = []
    
    # open the pdf file
    doc = fitz.open(file_path)

    # process individual pages
    for page in doc[1:]:
        
        # get content of the page
        text = page.get_text("text")
        
        # clean the content of page
        
        # replacements
        replacements = {'www.emestrada.org' : '',
                        'R  E  S  O  L  U  C  I  Ó  N': 'Resolucion',
                        '\n \n \n' : ''}

        for subs, rep in replacements.items():
            text =  text.replace(subs, rep)
        
        # remove whitespaces and create list of rows
        text = text.strip().replace('  ', ' ').split('\n')

        # extract exam details
        exam_details = text[-1]
        text = text[:-1]

        # extract only exercise statement
        times_a_appeared = 0
        for index, row in enumerate(text):
            if 'a)' in row:
                times_a_appeared += 1
                # it's the second occurrence of a)
                if times_a_appeared == 2:
                    text = text[index:]
                    break
        
        # clean relative masses
        for index, row in enumerate(text):
            if ('Masas atómicas relativas' in row) or ('Dato:' in row) :
                text = text[:index]
                break
        
        # remove the lines starting with just a number
        text = [i for i in text if len(i) > 1]
        
        # join the lines of the statement
        joined_text = ''
        for row in text:

            if row.startswith('b'):
                joined_text += '\n' + row
            else:
                joined_text += row

        # extract the year and exam
        try:
            exam = exam_details.split(' ')
            # extract subject
            subjects.append(exam[0].replace('.', '').title())
            
            years.append(exam[1].replace('.', ''))
            # get exam
            if exam[2] == 'RESERVA':
                exams.append(exam[2].title() +  ' ' + ' '.join(exam[3]))
            else:
                exams.append(exam[2].title())
            # get exercise
            if exam[2] == 'RESERVA':
                exercises.append(' '.join(exam[4:]).title())
            else:
                exercises.append(' '.join(exam[3:]).title())
        except:
            print(f'Problems with exercise data in {file_path.name}' \
                  f' on page {page.number + 1}.')
            print(exam_details)
                   
        statements.append(joined_text)
        pages.append(page.number + 1)

        # update the content with the page dict
        # content[f'{file_path.name}_page_{page.number + 1}'] = page_dict

    return {'subject' : subjects,   
            'year': years,
            'exam': exams,
            'exercise': exercises,
            'statement': statements,
            'page': pages}

In [200]:
page_dict = {}
exam_details = 'QUÍMICA. 2020. JUNIO. C4'
exam = exam_details.split(' ')
exam


['QUÍMICA.', '2020.', 'JUNIO.', 'C4']

In [201]:
# extract subject
page_dict['subject'] = (exam[0]
                        .replace('.', '')
                        .title()
)
page_dict['year'] = exam[1].replace('.', '')
# get exam
if exam[2] == 'RESERVA':
    page_dict['exam'] = exam[2].title() +  ' ' + ' '.join(exam[3])
else:
    page_dict['exam'] = exam[2].title()
# get exercise
if exam[2] == 'RESERVA':
    page_dict['exercise'] = ' '.join(exam[4:]).title()
else:
    page_dict['exercise'] = ' '.join(exam[3:]).title()

page_dict

{'subject': 'Química', 'year': '2020', 'exam': 'Junio.', 'exercise': 'C4'}

In [226]:
content = {}
topic = 'Redox'
for file in file_paths:
    content.update(extract_info(file))

Problems with exercise data in 2020 - Reacciones Redox.pdf on page 2.



In [231]:
content

{'subject': ['Química',
  'Química',
  'Química',
  'Química',
  'Química',
  'Química',
  'Química',
  'Química'],
 'year': ['2006', '2006', '2006', '2006', '2006', '2006', '2006', '2006'],
 'exam': ['Junio.',
  'Junio.',
  'Reserva 1 .',
  'Reserva 1 .',
  'Reserva 2 .',
  'Reserva 3 .',
  'Reserva 4 .',
  'Septiembre.'],
 'exercise': ['Ejercicio 3. Opción A',
  'Ejercicio 6. Opción B',
  'Ejercicio 3. Opción A',
  'Ejercicio 6. Opción B',
  'Ejercicio 5. Opción A',
  'Ejercicio 6. Opción B',
  'Ejercicio 3. Opción B',
  'Ejercicio 5. Opción A'],
 'statement': ['a) El número de oxidación del oxígeno pasa de  a 0. \nb) El carbono se oxida a CO c) El HNO se reduce a NO. ',
  'a) ¿Qué intensidad de corriente ha pasado a través de la cuba electrolítica?. \nb) Calcule la molaridad de la disolución inicial de AgNO . Datos: F=96500 C. Masas atómicas: Ag108 ; N14 ; O16',
  'a) Un elemento se reduce cuando pierde electrones. \nb) Una especie química se oxida al mismo tiempo que otra se reduce

In [230]:
df = pd.DataFrame(content)
df.shape

(8, 6)

In [ ]:
def save_dict_to_csv(data, filename):
    
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        # Write the header
        writer.writerow(data.keys())
        
        # Write the values
        writer.writerow(data.values())

In [221]:
# export to csv
save_dict_to_csv(content.values(), f'{topic}.csv')

AttributeError: 'dict_values' object has no attribute 'keys'

In [215]:
df = pd.read_csv('Redox.csv')

In [216]:
df.head()

,2005 - Reacciones Redox.pdf_page_2,2005 - Reacciones Redox.pdf_page_3,2005 - Reacciones Redox.pdf_page_4,2005 - Reacciones Redox.pdf_page_5,2005 - Reacciones Redox.pdf_page_6,2005 - Reacciones Redox.pdf_page_7,2005 - Reacciones Redox.pdf_page_8,2005 - Reacciones Redox.pdf_page_9,2008 - Reacciones Redox.pdf_page_2,2008 - Reacciones Redox.pdf_page_3,...,2010 - Reacciones Redox.pdf_page_8,2010 - Reacciones Redox.pdf_page_9,2006 - Reacciones Redox.pdf_page_2,2006 - Reacciones Redox.pdf_page_3,2006 - Reacciones Redox.pdf_page_4,2006 - Reacciones Redox.pdf_page_5,2006 - Reacciones Redox.pdf_page_6,2006 - Reacciones Redox.pdf_page_7,2006 - Reacciones Redox.pdf_page_8,2006 - Reacciones Redox.pdf_page_9
0,"{'subject': 'Química', 'year': '2005', 'exam':...","{'subject': 'Quimica', 'year': '2005', 'exam':...","{'subject': 'Quimica', 'year': '2005', 'exam':...","{'subject': 'Quimica', 'year': '2005', 'exam':...","{'subject': 'Quimica', 'year': '2005', 'exam':...","{'subject': 'Quimica', 'year': '2005', 'exam':...","{'subject': 'Quimica', 'year': '2005', 'exam':...","{'subject': 'Química', 'year': '2005', 'exam':...","{'subject': 'Química', 'year': '2008', 'exam':...","{'subject': 'Química', 'year': '2008', 'exam':...",...,"{'subject': 'Química', 'year': '2010', 'exam':...","{'subject': 'Química', 'year': '2010', 'exam':...","{'subject': 'Química', 'year': '2006', 'exam':...","{'subject': 'Química', 'year': '2006', 'exam':...","{'subject': 'Química', 'year': '2006', 'exam':...","{'subject': 'Química', 'year': '2006', 'exam':...","{'subject': 'Química', 'year': '2006', 'exam':...","{'subject': 'Química', 'year': '2006', 'exam':...","{'subject': 'Química', 'year': '2006', 'exam':...","{'subject': 'Química', 'year': '2006', 'exam':..."


In [209]:
pd.DataFrame(content.values(), columns=['subject', 'year', 'exam', 'exercise', 'statement', 'page'])

,subject,year,exam,exercise,statement,page
0,Química,2024,Junio,Ejercicio C4,a) Ajuste las ecuaciones iónica y molecular po...,2
1,Química,2024,Reserva 1,Ejercicio C4,a) Ajuste las ecuaciones iónica y molecular po...,3
2,Química,2024,Reserva 2,Ejercicio B4,"a) Justifique, escribiendo la semirreacción qu...",4
3,Química,2024,Reserva 2,Ejercicio C4,a) Ajuste las ecuaciones iónica y molecular po...,5
4,Química,2024,Reserva 3,Ejercicio B5,a) 2KMnO8H SO5K C O2MnSO8H O10CO6K SO\nb) CaCO...,6
5,Química,2024,Reserva 3,Ejercicio C2,a) El tiempo necesario para que todo el cobre ...,7
6,Química,2024,Reserva 4,Ejercicio B5,"a) Indique, razonadamente, cuál es el cátodo y...",8
7,Química,2024,Reserva 4,Ejercicio C4,a) Los gramos de calcio que se depositarán si ...,9
8,Química,2024,Julio,Ejercicio C4,a) Ajuste las ecuaciones iónica y molecular po...,10


In [150]:
'EJERCICIO C4'.replace('EJERCICIO', 'Ejercicio')

'Ejercicio C4'

In [129]:
content

{'page_2': {'subject': 'QUÍMICA',
  'year': '2024',
  'exam': 'JUNIO',
  'exercise': 'EJERCICIO C4',
  'page_content': 'a) Ajuste las ecuaciones iónica y molecular por el método del ión-electrón. \nb) Calcule el volumen de Cl obtenido a 0ºC y 1 atm de presión a partir de 30 mL de una disolución 0’5 M de KMnO y 50 mL de una disolución 0’25 M de HCl. ',
  'page_number': 2},
 'page_3': {'subject': 'QUÍMICA',
  'year': '2024',
  'exam': 'RESERVA 1',
  'exercise': 'EJERCICIO C4',
  'page_content': "a) Ajuste las ecuaciones iónica y molecular por el método del ion-electrón. \nb) Calcule la masa de níquel que podrá oxidarse con 1 mL de ácido nítrico comercial del 70% de riqueza en masa y densidad 1'42 g mL −. ",
  'page_number': 3},
 'page_4': {'subject': 'QUÍMICA',
  'year': '2024',
  'exam': 'RESERVA 2',
  'exercise': 'EJERCICIO B4',
  'page_content': "a) Justifique, escribiendo la semirreacción que tiene lugar en cada uno, cuál es el ánodo y cuál el cátodo. \nb) Calcule el potencial estánd

In [134]:
for exam in content.values():
    print(v['page_content'])
    print('\n' + '*' * 10 + '\n')

a) Ajuste las ecuaciones iónica y molecular por el método del ion-electrón. 
b) ¿Cuántos gramos de Cr(OH) del 90% de riqueza se necesitan para reaccionar completamente con 50 mL de una disolución de KClO 0,55 M? 

**********

a) Ajuste las ecuaciones iónica y molecular por el método del ion-electrón. 
b) ¿Cuántos gramos de Cr(OH) del 90% de riqueza se necesitan para reaccionar completamente con 50 mL de una disolución de KClO 0,55 M? 

**********

a) Ajuste las ecuaciones iónica y molecular por el método del ion-electrón. 
b) ¿Cuántos gramos de Cr(OH) del 90% de riqueza se necesitan para reaccionar completamente con 50 mL de una disolución de KClO 0,55 M? 

**********

a) Ajuste las ecuaciones iónica y molecular por el método del ion-electrón. 
b) ¿Cuántos gramos de Cr(OH) del 90% de riqueza se necesitan para reaccionar completamente con 50 mL de una disolución de KClO 0,55 M? 

**********

a) Ajuste las ecuaciones iónica y molecular por el método del ion-electrón. 
b) ¿Cuántos gramos 

In [135]:
content['page_7']['page_content']

'a) El tiempo necesario para que todo el cobre contenido en 250 mL de una disolución acuosa 0,1 M de iones Cu+ se deposite como cobre metálico, cuando se hace pasar una corriente eléctrica de 1,5 A. \nb) La intensidad de corriente eléctrica que se debe hacer pasar a través de una disolución acuosa de iones Au+ , si se quiere obtener 1 g de oro metálico en 30 minutos. Datos: 96500 C mol −'